# Laboratório 5 – Aplicação de RNN para Compreensão de Linguagem Natural

Obs: Esse laboratório foi baseado [nesse artigo](https://chsasank.github.io/spoken-language-understanding.html)

O problema que vamos lidar aqui é uma das tarefas típicas da [compreensão de linguagem natural](https://en.wikipedia.org/wiki/Natural_language_understanding). Nosso objetivo é identificar elementos principais em uma frase. A tarefa será, dada uma pergunta feita em um sistema sobre vôos, identificar elementos como cidade de origem e destino, datas, horários, etc.

O conjunto de dados utilizado é o Airline Travel Information System (ATIS). Esses dados foram coletados na década de 90 pelo Departamento de defesa americano (DARPA). ATIS consiste em pares de perguntas e de labels para cada palavra da pergunta. 

Aqui está um exemplo de pergunta e seus rótulos, que estão codificados usando a notação [Inside Outside Beginning (IOB)](https://en.wikipedia.org/wiki/Inside_Outside_Beginning):


Primeiro importamos as bibliotecas que vamos usar e os dados de um arquivo no formato [pickle](https://docs.python.org/3/library/pickle.html).

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pickle
import numpy as np
import progressbar

from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.preprocessing import MultiLabelBinarizer

from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN, GRU, LSTM
from keras.layers.core import Dense, Dropout
from keras.layers.wrappers import TimeDistributed
from keras.layers import Convolution1D, MaxPooling1D
from keras import optimizers
from keras.regularizers import l1_l2

### Carregando e analisando os dados

In [ ]:
with open('../input/atis.pkl/atis.pkl', 'rb') as f:
    train_set, valid_set, test_set, dicts = pickle.load(f, encoding='latin1')

In [ ]:
train_x, train_ne, train_label = train_set
val_x, val_ne, val_label = valid_set
test_x, test_ne, test_label = test_set

A variável dicts guarda os dicionários de palavras para índices. Aqui criamos os índices inversos (números para palavras).

In [ ]:
w2idx, ne2idx, labels2idx = dicts['words2idx'], dicts['tables2idx'], dicts['labels2idx']

# Create index to word/label dicts
idx2w  = {w2idx[k]:k for k in w2idx}
idx2ne = {ne2idx[k]:k for k in ne2idx}
idx2la = {labels2idx[k]:k for k in labels2idx}

Cada exemplo é um vetor com os índices das palavras no dicionário. Para obter as palavras usamos o índice inverso, como ilustrado abaixo com o exemplo 0.

In [ ]:
print(train_x[0])
print([idx2w[i] for i in train_x[0]])

Veja os labels do exemplo 0.

In [ ]:
print([idx2la[i] for i in train_label[0]])

Aqui transformamos dados de treino e validação que guardam os índices, em listas com as palavras.

In [ ]:
words_val = [ list(map(lambda x: idx2w[x], w)) for w in val_x]
groundtruth_val = [ list(map(lambda x: idx2la[x], y)) for y in val_label]
words_train = [ list(map(lambda x: idx2w[x], w)) for w in train_x]
groundtruth_train = [ list(map(lambda x: idx2la[x], y)) for y in train_label]

Note que estamos lidando com um conjunto de dados muito pequeno. São apenas 4978 exemplos para treinamento e 572 palavras distintas nas perguntas.

In [ ]:
n_classes = len(idx2la)
n_vocab = len(idx2w)
n_examples = len(words_train)
print('#labels: ', n_classes, '\t#distinct words: ', n_vocab, '\t#examples: ', n_examples)

### Criando a RNN e realizando o treinamento e validação

Aqui está nosso código que vai criar nosso modelo de uma rede neuronal recorrente. Usaremos a classe GRU do keras, que é um tipo especial de rede recorrente.

In [ ]:
model = Sequential()
model.add(Embedding(n_vocab,100))

## Essas duas camadas são opcionais. Daremos mais detalhes nos próximos laboratórios
#model.add(Convolution1D(64, 5, border_mode='same', activation='relu'))
#model.add(Dropout(0.1))

model.add(GRU(100, return_sequences=True, 
              kernel_regularizer=l1_l2(l1=0.0, l2=0.0), 
              recurrent_regularizer=l1_l2(l1=0.0, l2=0.0)
             ))

model.add(TimeDistributed(Dense(n_classes, activation='softmax')))
#optm = optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
#optm = optimizers.RMSprop(lr=0.01, rho=0.9, epsilon=None, decay=0.0)
optm = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy', optimizer=optm)

No código abaixo treinamos a nossa rede recorrente. O número de épocas pode ser ajustado. A cada época é consolidado as métricas `precision`, `recall` e `f1-score`. Ver detalhes de como é feito o cálculo [aqui](https://en.wikipedia.org/wiki/F1_score). Obtendo 1 nestes scores temos um classificador perfeito.

In [ ]:
ml = MultiLabelBinarizer(classes=list(idx2la.values())).fit(idx2la.values())

def conlleval( trues, preds ):
    trues = ml.transform(trues)
    preds = ml.transform(preds)
    return score(trues, preds, beta=1, average='macro' )


In [ ]:
# Defina aqui o número de épocas
n_epochs = 12

train_f_scores = []
val_f_scores = []
best_val_f1 = 0
con_dict = {}

for i in range(n_epochs):
    print("\nEpoch {}".format(i))
    
    print("Training =>")
    train_pred_label = []
    avgLoss = 0
    
    bar = progressbar.ProgressBar(max_value=len(train_x))
    for n_batch, sent in bar(enumerate(train_x)):
        label = train_label[n_batch]
        label = np.eye(n_classes)[label][np.newaxis,:]
        sent = sent[np.newaxis,:]
        
        if sent.shape[1] > 1: #some bug in keras
            loss = model.train_on_batch(sent, label)
            avgLoss += loss

        pred = model.predict_on_batch(sent)
        pred = np.argmax(pred,-1)[0]
        train_pred_label.append(pred)

    avgLoss = avgLoss/n_batch
    
    predword_train = [ list(map(lambda x: idx2la[x], y)) for y in train_pred_label]
    con_dict['p'], con_dict['r'], con_dict['f1'], _ = conlleval(groundtruth_train, predword_train)
    train_f_scores.append(con_dict['f1'])
    
    print('Loss = {}, Precision = {}, Recall = {}, F1 = {}'.format(avgLoss, con_dict['p'], con_dict['r'], con_dict['f1']))
    
    print("\n\nValidating =>")
    
    val_pred_label = []
    avgLoss = 0
    
    bar = progressbar.ProgressBar(max_value=len(val_x))
    for n_batch, sent in bar(enumerate(val_x)):
        label = val_label[n_batch]
        label = np.eye(n_classes)[label][np.newaxis,:]
        sent = sent[np.newaxis,:]
        
        if sent.shape[1] > 1: #some bug in keras
            loss = model.test_on_batch(sent, label)
            avgLoss += loss

        pred = model.predict_on_batch(sent)
        pred = np.argmax(pred,-1)[0]
        val_pred_label.append(pred)

    avgLoss = avgLoss/n_batch
    
    predword_val = [ list(map(lambda x: idx2la[x], y)) for y in val_pred_label]
    con_dict['p'], con_dict['r'], con_dict['f1'], _ = conlleval(groundtruth_val, predword_val)
    val_f_scores.append(con_dict['f1'])
    
    print('Loss = {}, Precision = {}, Recall = {}, F1 = {}'.format(avgLoss, con_dict['p'], con_dict['r'], con_dict['f1']))

    if con_dict['f1'] > best_val_f1:
        best_val_f1 = con_dict['f1']
        open('model_architecture.json','w').write(model.to_json())
        model.save_weights('best_model_weights.h5',overwrite=True)
        print("Best validation F1 score = {}".format(best_val_f1))



In [ ]:
print("Best epoch to stop = {}".format(val_f_scores.index(max(val_f_scores))))
print("Best validation F1 score = {}".format(best_val_f1))

Aqui escolhemos um exemplo aleatório dos dados de validação para comparar os labels reais x previstos

In [ ]:
idx = np.random.randint(len(words_val))
preds = [idx2la[i[0]] for i in np.argmax(model.predict(val_x[idx]), -1)]
for k in zip(words_val[idx],groundtruth_val[idx], preds):
    print('Word: %s\t\tLabel: %s\t\tPred: %s' % k)